In [2]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
nltk.download('stopwords')
print(stopwords.words('english'))
stopwords_eng = stopwords.words('english')
translator = str.maketrans('', '', string.punctuation)
mention_pattern = re.compile(r'@\w+')
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def pre_process(tweet):
    # remove punctuations
    tweet = tweet.translate(translator)

    # remove rt text
    tweet2 = tweet.strip()
    tweet2 = re.sub(r'^RT[\s]+', '', tweet2)

    # remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)
    tweet2 = re.sub(r'#', '', tweet2)

    tweet2 = re.sub(mention_pattern, '', tweet2)

    # single numeric
    tweet2 = re.sub(r'[0-9]', '', tweet2)

    # tokenizing
    tweet_tokens = tokenizer.tokenize(tweet2)
    # print(tweet_tokens)

    # removing stop words and stemming
    processed_tweet_tokens = []
    for word in tweet_tokens:
        if word not in stopwords_eng:
            processed_tweet_tokens.append(word)
    result_tweet = ' '.join(processed_tweet_tokens)

    return result_tweet

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
data = pd.read_csv('/content/sample_data/Sentiment Analysis Dataset.csv', error_bad_lines=False)

<ipython-input-16-14341faeebb7>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/sample_data/Sentiment Analysis Dataset.csv', error_bad_lines=False)
Skipping line 8836: expected 4 fields, saw 5

Skipping line 535882: expected 4 fields, saw 7



In [17]:
# pre_process(data['SentimentText'][3])
data['SentimentText'] =  data['SentimentText'].apply(pre_process)
data

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,sad apl friend
1,2,0,Sentiment140,missed new moon trailer
2,3,1,Sentiment140,omg already
3,4,0,Sentiment140,omgaga im sooo im gunna cry ive dentist since ...
4,5,0,Sentiment140,think mi bf cheating tt
...,...,...,...,...
1578607,1578623,1,Sentiment140,zzzz finally night tweeters
1578608,1578624,1,Sentiment140,zzzz sleep well people
1578609,1578625,0,Sentiment140,zzzzzzzzzz wait homework
1578610,1578626,0,Sentiment140,zzzzzzzzzzzz meh


In [18]:
x = data.iloc[:, [-1]]
y = data.iloc[:, [1]]


In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [20]:
x_train

,SentimentText
1329183,real peter cetera please wave cell phone sing ...
49935,ashninaa mad late craaazzzyyy lol missed u baby
1536731,sorry quiet afternoon feeling bit
1226652,hacked account aim make new one
1357391,xxkassyxx haha keep protien level guess dunno ...
...,...
1172794,really want go movie startrek drag hell else
976945,reading new moon cant wait get happy part sad ...
1036052,markhoppus itll still top mark
1106465,problemonever forgot youthought disappeared mi...


In [22]:
vectorizer = CountVectorizer(max_features=1000)
vectorizer.fit(x_train['SentimentText'])
x_train_vector = vectorizer.transform(x_train['SentimentText'])
x_test_vector = vectorizer.transform(x_test['SentimentText'])

In [23]:
print(x_train.shape)
print(x_train_vector.shape)
print(x_test.shape)
print(x_test_vector.shape)

(1262889, 1)
(1262889, 1000)
(315723, 1)
(315723, 1000)


Naive Bayes

In [ ]:
gnb = GaussianNB()
model = GaussianNB()
model.fit(x_train_vector.toarray(), y_train.values.ravel())
y_predict = model.predict(x_test_vector.toarray())


In [ ]:
bayes_accuracy = accuracy_score(y_test, y_predict)
print('accuracy ' + str(bayes_accuracy))

accuracy 0.6951441611792616


In [ ]:
# confusion matrix
cm = confusion_matrix(y_test, y_predict)
cm

array([[ 92928,  64981],
       [ 31269, 126545]], dtype=int64)

SVM

In [11]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(x_train_vector.toarray(), y_train.values.ravel())
y_predict_svc = svc.predict(x_test_vector.toarray())

In [12]:
svc_accuracy = accuracy_score(y_test, y_predict_svc)
print(svc_accuracy)
cm = confusion_matrix(y_test, y_predict_svc)
cm

0.7621428571428571


array([[780, 109],
       [224, 287]])